In [86]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [87]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash', google_api_key=os.getenv('GEMINI_KEY'))
embeddings= GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=os.getenv('GEMINI_KEY'))

In [88]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableMap
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_community.vectorstores import Chroma
from langchain .text_splitter import RecursiveCharacterTextSplitter

In [89]:
loader = PyPDFLoader('./agents.pdf')
text = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
final_docs = text_splitter.split_documents(text)

vector_db = Chroma.from_documents(final_docs, embeddings )
retriever=  vector_db.as_retriever(search_kwargs={"k":3})

In [90]:
query = "what is agents  "
docs = retriever.invoke(query)
print(docs[0].metadata)
print(docs[1].page_content)
for doc in docs:
    print(doc)

{'creationdate': '2025-02-14T12:31:53+05:00', 'creator': 'Adobe InDesign 19.1 (Windows)', 'moddate': '2025-02-14T12:33:59+05:00', 'page': 9, 'page_label': '10', 'producer': 'Adobe PDF Library 17.0', 'source': './agents.pdf', 'total_pages': 93, 'trapped': '/False'}
provides a structured approach to defining and managing agents. It’s very straightforward
page_content='agents along with their characteristics, examples, and when you can use them.' metadata={'creationdate': '2025-02-14T12:31:53+05:00', 'creator': 'Adobe InDesign 19.1 (Windows)', 'moddate': '2025-02-14T12:33:59+05:00', 'page': 9, 'page_label': '10', 'producer': 'Adobe PDF Library 17.0', 'source': './agents.pdf', 'total_pages': 93, 'trapped': '/False'}
page_content='provides a structured approach to defining and managing agents. It’s very straightforward' metadata={'creationdate': '2025-02-14T12:31:53+05:00', 'creator': 'Adobe InDesign 19.1 (Windows)', 'moddate': '2025-02-14T12:33:59+05:00', 'page': 30, 'page_label': '31', 'p

In [91]:
def function1(AgentState):
    message = AgentState["messages"]
    question = message[-1]

    complete_prompt = "Your Tasks is provide only the brief answer based on the user query.\
        Dont include too much reasoning. Following the user query {question}"

    response= llm.invoke(complete_prompt)
    AgentState['messages'].append(response.content)
    print(AgentState)
    return AgentState

In [92]:
def function2(AgentState):
    messgae= AgentState['messages']
    question = messgae[0]
    template = """Answer the following question based on the context
    {context}

    Question: {question}
    """
    prompt = ChatPromptTemplate.from_template(template)
    
    retreival_chain=(
        {'context':retriever, 'question':RunnablePassthrough()}
        |prompt
        |llm
        |StrOutputParser()
    )
    result = retreival_chain.invoke(question)
    return result


In [93]:
from langgraph.graph import Graph
workflow = Graph()
workflow.add_node('llm', function1)
workflow.add_node('rag', function2)
workflow.add_edge('llm', 'rag')
workflow.set_entry_point('llm')
workflow.set_finish_point('rag')
app = workflow.compile()

In [94]:
inputs = {'messages':['tell me about agents ']}

In [95]:
for output in app.stream(inputs):
    for key,value in output.items():
        print(f'here is the output from {key}')
        print('-------')
        print(value)
        print('\n')

{'messages': ['tell me about agents ', 'Okay, I understand. I will provide brief answers, avoiding excessive reasoning.']}
here is the output from llm
-------
{'messages': ['tell me about agents ', 'Okay, I understand. I will provide brief answers, avoiding excessive reasoning.']}


here is the output from rag
-------
Agents have characteristics and examples, and there are specific times when you can and cannot use them.




In [96]:
loader = TextLoader('indian_economy.txt')
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=190, chunk_overlap=50)
final_docs = text_splitter.split_documents(docs)

db2 = Chroma.from_documents(final_docs, embeddings)
retreiver2 =db2.as_retriever(search_kwargs={'k':3})



In [97]:
query= "tell me about Economic Reforms "
docs  = retreiver2.invoke(query)
print(docs[0].page_content)

🔹 Economic Reforms:
1991 Liberalization: Opened the economy to global markets, reduced state control, and encouraged foreign investment.


In [98]:
from typing import Annotated, Sequence, TypedDict
import operator
from langchain_core.messages import BaseMessage
from langchain.prompts import PromptTemplate

In [99]:
class AgentState(TypedDict):
    messages :Annotated[Sequence[BaseMessage], operator.add]

In [100]:
from pydantic import BaseModel, Field
class TopicSelection(BaseModel):
    topic:str=Field(description='Selected Topic')
    Reasoning:str=Field(description='Reasoning behind the toipic seelction')

In [101]:
from langchain.output_parsers import PydanticOutputParser
parser = PydanticOutputParser(pydantic_object=TopicSelection)

In [124]:
def function1(state):
    message = state['messages']
    question = message[-1]
    print(question)

   
    template="""
    Your task is to classify the given user query into one of the following categories: [India, Sports, Not Related]. 
    Only respond with the category name and nothing else.
    

    User query: {question}
    {format_instructions}
    """
    prompt = PromptTemplate(template=template,
                            input_variables=[question],
                            partial_variables={
                                'format_instructions':parser.get_format_instructions()
                            })
    chain = prompt|llm|parser
    resposne = chain.invoke({'question':question, 'format_instructions':parser.get_format_instructions()})
    print(resposne)
    return {'messages':[resposne.topic]}

In [103]:
state = {"messages":["tell me about indian economy"]}
function1(state)

tell me about indian economy
topic='India' Reasoning="The query explicitly mentions 'indian economy', which directly relates to India."


{'messages': ['India']}

In [116]:
def function2(state):
    messages = state['messages']
    question = messages[0]

    template = """Answer the question based only on the following context:
    {context}

    Question: {question}"""
    prompt= ChatPromptTemplate.from_template(template)
    retrevial_chain = (
        RunnableMap({'context':retreiver2, "question":RunnablePassthrough() })
        |prompt
        |llm
        |StrOutputParser()
    )
    result = retrevial_chain.invoke(question)
    return {'messages':[result]}

In [117]:
def function_3(state):
    print('-> Calling LLM ->')

    messages = state['messages']
    print(messages)
    question = messages[0] ## Fetching the user question

    # Normal LLM call
    complete_query = "Answer the follow question with you knowledge of the real world. Following is the user question: " + question
    response = llm.invoke(complete_query)
    return {"messages": [response.content]}

In [118]:
state={"messages":["tell me about agents in langraph"]}
function_3(state)

-> Calling LLM ->
['tell me about agents in langraph']


{'messages': ['Okay, let\'s talk about Agents in the context of LangGraph.\n\nIt\'s important to clarify that LangGraph itself is a *framework for building stateful, multi-actor applications*. It\'s *not* a pre-built agent implementation like you might find in LangChain or AutoGen. LangGraph provides the *infrastructure* to build sophisticated agent systems, but you still need to define the specific agents and their behaviors.\n\nHere\'s a breakdown of how agents fit into the LangGraph world:\n\n**1. What is an Agent (in this context)?**\n\nIn the context of LangGraph, an "agent" is essentially a *node* within your graph that represents an autonomous entity. This entity:\n\n*   **Receives Input (State):**  It takes in a piece of the graph\'s current state. This state could contain:\n    *   User queries.\n    *   Results from previous agent actions.\n    *   Contextual information (e.g., current date/time, data from a database).\n*   **Processes the Input:** It uses some logic (often i

In [119]:
def router(state):
    print("---router state")

    messages = state['messages']
    last_messages =messages[-1]
    print(last_messages)
    if 'India' in last_messages:
        return "RAG_CALL"
    else:
        return "LLM_CALL"

In [120]:
from langgraph.graph import StateGraph, END


workflow5 = StateGraph(AgentState)
workflow5.add_node('agent', function1)
workflow5.add_node('Rag', function2)
workflow5.add_node('llm', function_3)

workflow5.set_entry_point('agent')
workflow5.add_conditional_edges(
    'agent',
    router,
    {
        "RAG_CALL":"Rag",
        "LLM_CALL":'llm'
    }
)
workflow5.add_edge('Rag', END)
workflow5.add_edge('llm', END)

app = workflow5.compile()

In [126]:
result = app.invoke({'messages':['who is the president of USa  ']})
result

who is the president of USa  
topic='Not Related' Reasoning='The query is about the president of the USA, which is not related to India or Sports.'
---router state
Not Related
-> Calling LLM ->
['who is the president of USa  ', 'Not Related']


{'messages': ['who is the president of USa  ',
  'Not Related',
  'The President of the United States is Joe Biden.']}